<a href="https://colab.research.google.com/github/wjdrnqja9/TIL/blob/main/Flask/SimpleProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 참고: https://hleecaster.com/ml-linear-regression-example/

## 라이브러리 설치, 호출

!pip3 install -U scikit-learn<br>
!pip3 install pandas<br>
!pip3 install numpy<br>
!pip3 install matplotlib<br>
!pip3 install statsmodels<br>

In [ ]:
!pip3 install -U scikit-learn
!pip3 install pandas
!pip3 install numpy
!pip3 install matplotlib
!pip3 install statsmodels

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

## 데이터 다운로드 (특별할인 판매)

In [ ]:
import pandas as pd
df = pd.read_csv('diabetes.csv')
print(df.head())
print(df.shape)

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
(768, 9)


## Input, Feature 설정

In [ ]:
Label = df['Outcome']
InputFeature = df[["Pregnancies", "Glucose", "BloodPressure", "SkinThickness", 
                   "Insulin", "BMI", "DiabetesPedigreeFunction", "Age"]]

In [ ]:
InputFeature.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


## Keras Logit 모델 fitting

In [ ]:
model = Sequential()
model.add(Dense(5, activation='linear', input_shape=(8,) ))
model.add(Dense(10, activation='linear'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

## Keras 모델 살펴보기

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 45        
                                                                 
 dense_1 (Dense)             (None, 10)                60        
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 116
Trainable params: 116
Non-trainable params: 0
_________________________________________________________________


## Call-back 함수
## 모델 학습

In [ ]:
# Call-back 함수
# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='Models/{epoch:03d}-{loss:.4f}-{accuracy:.4f}.hdf5',
            monitor='loss', verbose=1, save_best_only=True, mode='min')

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='loss',factor=0.8,patience=3, verbose=1, min_lr=1e-8)

CALLBACK = [CP, LR]

In [ ]:
# model.fit(x=InputFeature, y=Label, epochs=100, shuffle=True, batch_size=3, callbacks=CALLBACK)

# Model Load 하기 전, hdf5 파일 이름 꼭 확인하기

In [ ]:
model.load_weights("Models/100-0.4767-0.7773.hdf5")

## FLASK 셋팅하기

In [ ]:
from flask import Flask
from flask import render_template
from flask import request

In [ ]:
app = Flask(__name__)

# FLASK API 구현부분

In [ ]:
@app.route('/')
@app.route('/Diabetes')
def DiabetesPrediction():
    Pregnancies = request.args.get("Pregnancies")
    Glucose = request.args.get("Glucose")
    BloodPressure = request.args.get("BloodPressure")
    SkinThickness = request.args.get("SkinThickness")
    Insulin = request.args.get("Insulin")
    BMI = request.args.get("BMI")
    DiabetesPedigreeFunction = request.args.get("DiabetesPedigreeFunction")
    Age = request.args.get("Age")
       
    if Pregnancies == None or Glucose == None:
        return render_template('Diabetes.html', Output = '')
    
    Input = pd.DataFrame({
        'Pregnancies': [ float(Pregnancies) ],
        'Glucose': [ float(Glucose) ],
        'BloodPressure': [ float(BloodPressure) ],
        'SkinThickness': [ float(SkinThickness) ],
        'Insulin': [ float(Insulin) ],
        'BMI': [ float(BMI) ],
        'DiabetesPedigreeFunction': [ float(DiabetesPedigreeFunction) ],
        'Age': [float(Age)]
    })
    ModelOutput = model.predict(Input)[0][0]

    return render_template('Diabetes.html', Output = ModelOutput)

# Flask, port 5000으로 실행

In [ ]:
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.2.16:5000 (Press CTRL+C to quit)
127.0.0.1 - - [08/Jul/2022 09:36:17] "GET /SpecialSale?Pregnancies=1&Glucose=75&BloodPressure=100&SkinThickness=75&Insulin=30&BMI=30&DiabetesPedigreeFunction=10&Age=33 HTTP/1.1" 404 -
127.0.0.1 - - [08/Jul/2022 09:36:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2022 09:36:22] "GET /static/image.png HTTP/1.1" 304 -


1/1 [==============================] - 0s 207ms/step


127.0.0.1 - - [08/Jul/2022 09:36:30] "GET /Diabetes?Pregnancies=1&Glucose=50&BloodPressure=30&SkinThickness=30&Insulin=30&BMI=30&DiabetesPedigreeFunction=25&Age=33 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2022 09:36:31] "GET /static/image.png HTTP/1.1" 304 -


1/1 [==============================] - 0s 25ms/step


127.0.0.1 - - [08/Jul/2022 09:39:34] "GET /Diabetes?Pregnancies=1&Glucose=50&BloodPressure=30&SkinThickness=30&Insulin=30&BMI=30&DiabetesPedigreeFunction=25&Age=33 HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2022 09:39:35] "GET /static/image.png HTTP/1.1" 304 -


In [ ]:
# http://127.0.0.1:5000/SpecialSale/1/38